In [ ]:
import torch

In [36]:
import numpy.random as random

In [41]:
import matplotlib.pyplot as plt

In [ ]:
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [ ]:
print(category_lines)

In [ ]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [ ]:
print(letterToTensor('J'))
print(letterToTensor('J').shape)

In [ ]:
print(lineToTensor('Jones').shape)

In [38]:
def random_train_sample(category_lines, all_categories):
    def random_choice(a):
        random_index = random.randint(0,len(a) - 1)
        return a[random_index]
    category = random_choice(all_categories)
    line = random_choice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)],dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

## Creating RNN from Scratch

In [10]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        #self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_tensor, hidden_tensor):
        combined = torch.cat((input_tensor, hidden_tensor), 1)
        
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        
        return output, hidden
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

#perform one step
input_tensor = letterToTensor('A') #input one hot vector
hidden_tensor = rnn.initHidden() # initialize h with zeros

output, next_hidden = rnn(input_tensor, hidden_tensor)

print(output.size())
print(next_hidden.size())


In [ ]:
#apply this on a word (sequence of letters)
input_tensor = lineToTensor('verma')
hidden_tensor = rnn.initHidden()

output, next_hidden = rnn(input_tensor[0], hidden_tensor)#works for 1 letter.
print(input_tensor[0].size())
print(output.size())
print(next_hidden.size())


In [20]:
#convert output to categorical
def category_from_output(output):
    category_index = torch.argmax(output).item()
    return all_categories[category_index]

In [ ]:
print(category_from_output(output))

In [ ]:
print(input_tensor.size())

In [22]:
criterion = nn.NLLLoss()
learning_rate = 0.005
optimizer = torch.optim.SGD(rnn.parameters(), learning_rate)

In [28]:
# training loop
def train(line_tensor, category_tensor):
    hidden = rnn.initHidden()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return output, loss.item()

In [29]:
current_loss = 0
all_losses =[]
plot_steps, print_steps = 1000, 5000
epochs = 100000

In [ ]:
for i in range(epochs):
    #for every epoch, fetch random training sample
    category, line, category_tensor, line_tensor = random_train_sample(category_lines, all_categories)
    
    output, loss = train(line_tensor, category_tensor)
    current_loss += loss
    
    if (i+1) % plot_steps ==0:
        all_losses.append(current_loss / plot_steps)
        current_loss = 0
    
    

In [ ]:
plt.figure()
plt.plot(all_losses)
plt.show()

In [ ]:
#generate prediction
def predict(input_line):
    print(f"\n {input_line}")
    with torch.no_grad():
        line_tensor = lineToTensor(input_line)
        hidden = rnn.initHidden()
        for i in range(line_tensor.size()[0]):
            output, hidden = rnn(line_tensor[i], hidden)
        
        prediction = category_from_output(output)
        print(prediction)
          
while True:
    sentence = input("Input")
    if sentence =="quit":
        break
    predict(sentence)